In [2]:
!pip install matplotlib-venn

In [3]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [4]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 123105 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.4.0-2ubuntu1.2_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.4.0-2ubuntu1.2) ...
Setting up libarchive-dev:amd64 (3.4.0-2ubuntu1.2) ...
Processing triggers for man-db (2.9.1-1) ...
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 13.9 MB/s eta 0:00:00
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31629 sha256=47c6eeb5f1408c2f53b883d31109cd25d5932021a7b27e219b9047b5bcd10992
  Stored in directory: /root/.cache/pip/wheels/3a/94/d0/6cd83c8a80a4236fd4cb2a1fd846ecf72ab1e0ac238c5951c0
Successfully built libarchive


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [9]:
from keras.preprocessing.text import Tokenizer

from keras.models import Sequential
from keras.layers import Embedding, LSTM, GRU, Bidirectional, Dense, Dropout
from keras.callbacks import EarlyStopping


In [6]:
from keras.utils import pad_sequences

In [10]:
data = pd.read_csv("urdu-sentiment-corpus-v1.tsv", sep="\t")


In [11]:
data.head()

,Tweet,Class
0,میں نے ایٹم بم بنایا ھے ۔۔۔۔او بھائی ایٹم بمب ...,P
1,چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...,N
2,ٹویٹر کا خیال کیسے آیا ؟,O
3,"سرچ انجن گوگل کے نائب صدر نے فضا میں ، 130,000...",P
4,ابھی تک اسکی لہریں کبھی کبھی آ جاتی ہیں یار :أْ,P


In [13]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   1000 non-null   object
 1   Class   999 non-null    object
dtypes: object(2)
memory usage: 15.8+ KB


,Tweet,Class
count,1000,999
unique,999,3
top,اللہ جانے وے ماہی تیرا پیار کی اے دل دی اوداسی...,N
freq,2,499


In [15]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
data.isna().values.any()

True

In [19]:
data.drop(data[data['Class']=='O'].index,inplace = True)

In [26]:
from sklearn.preprocessing import LabelEncoder
data['Class']=LabelEncoder().fit_transform(data['Class'])
piece = Tokenizer()
piece.fit_on_texts(data['Tweet'])
seq = piece.texts_to_sequences(data['Tweet'])

In [29]:
vocablen = len(piece.word_index) + 1
max_len = max(len(s) for s in seq)
x = pad_sequences(seq, maxlen=max_len, padding='post')
y = data['Class']


In [30]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.25,random_state=101)

In [33]:
from tensorflow.keras.layers import Dense, Dropout,Flatten,LSTM,Bidirectional,SimpleRNN

In [34]:
dummy1 = Sequential()
em = Embedding(vocablen,100,input_length=max_len,trainable = True)
dummy1.add(em)
dummy1.add(SimpleRNN(100,return_sequences=True ,dropout=0.3))
dummy1.add(Dense(512,activation='tanh'))
dummy1.add(Flatten())
dummy1.add(Dense(1,activation='sigmoid'))
dummy1.compile(loss ="binary_crossentropy", optimizer = 'adam', metrics=["accuracy"])

In [35]:
dummy1.fit(xtrain,ytrain,batch_size=20,epochs = 10)

Epoch 1/10
37/37 [==============================] - 4s 34ms/step - loss: 0.7156 - accuracy: 0.5156
Epoch 2/10
37/37 [==============================] - 1s 33ms/step - loss: 0.3101 - accuracy: 0.8612
Epoch 3/10
37/37 [==============================] - 1s 33ms/step - loss: 0.0319 - accuracy: 0.9850
Epoch 4/10
37/37 [==============================] - 2s 59ms/step - loss: 0.0224 - accuracy: 0.9878
Epoch 5/10
37/37 [==============================] - 2s 47ms/step - loss: 0.0041 - accuracy: 0.9946
Epoch 6/10
37/37 [==============================] - 1s 38ms/step - loss: -0.0218 - accuracy: 0.9959
Epoch 7/10
37/37 [==============================] - 2s 48ms/step - loss: -0.0088 - accuracy: 0.9932
Epoch 8/10
37/37 [==============================] - 2s 41ms/step - loss: 0.0077 - accuracy: 0.9864
Epoch 9/10
37/37 [==============================] - 2s 40ms/step - loss: -2.0940e-04 - accuracy: 0.9891
Epoch 10/10
37/37 [==============================] - 1s 36ms/step - loss: 0.0061 - accuracy: 0.9864


In [36]:
prediction1 = dummy1.predict(xtest)

8/8 [==============================] - 0s 10ms/step


In [41]:
import numpy as np
prediction2 = np.argmax(prediction1,axis=1)
ac = accuracy_score(ytest,prediction2)
pres = precision_score(ytest,prediction2,average = 'macro')

In [44]:
f1 = f1_score(ytest,prediction2, average = 'macro')
rec = recall_score(ytest,prediction2, average = 'macro')

In [45]:
dataset = pd.DataFrame({
 'precision':[pres],
 'recall' :[rec],
 'accuracy':[ac],
 'F1_score':[f1]
})


In [46]:
dataset

,precision,recall,accuracy,F1_score
0,0.244898,0.5,0.489796,0.328767


In [47]:
#BiLSTM
dummy2 = Sequential()
em = Embedding(vocablen,100,input_length=max_len,trainable = True)
dummy2.add(em)
dummy2.add(Bidirectional(LSTM(100,return_sequences=True)))
dummy2.add(Dropout(0.3))
dummy2.add(Dense(2,activation='relu'))
dummy2.add(Flatten())
dummy2.add(Dense(1,activation='sigmoid'))
dummy2.compile(loss ="binary_crossentropy", optimizer = 'sgd', metrics=["accuracy"])

In [48]:
dummy2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 37, 100)           549800    
                                                                 
 bidirectional (Bidirectiona  (None, 37, 200)          160800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 37, 200)           0         
                                                                 
 dense_2 (Dense)             (None, 37, 2)             402       
                                                                 
 flatten_1 (Flatten)         (None, 74)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 75        
                                                      

In [49]:
dummy2.fit(xtrain,ytrain,batch_size=30,epochs = 10)


Epoch 1/10
25/25 [==============================] - 7s 81ms/step - loss: 0.6935 - accuracy: 0.4707
Epoch 2/10
25/25 [==============================] - 2s 80ms/step - loss: 0.6933 - accuracy: 0.4844
Epoch 3/10
25/25 [==============================] - 3s 114ms/step - loss: 0.6932 - accuracy: 0.5252
Epoch 4/10
25/25 [==============================] - 2s 87ms/step - loss: 0.6929 - accuracy: 0.5197
Epoch 5/10
25/25 [==============================] - 2s 81ms/step - loss: 0.6929 - accuracy: 0.5170
Epoch 6/10
25/25 [==============================] - 2s 73ms/step - loss: 0.6930 - accuracy: 0.5116
Epoch 7/10
25/25 [==============================] - 2s 76ms/step - loss: 0.6929 - accuracy: 0.5129
Epoch 8/10
25/25 [==============================] - 2s 73ms/step - loss: 0.6928 - accuracy: 0.5143
Epoch 9/10
25/25 [==============================] - 2s 96ms/step - loss: 0.6930 - accuracy: 0.5224
Epoch 10/10
25/25 [==============================] - 4s 170ms/step - loss: 0.6929 - accuracy: 0.5143


In [51]:
prediction1 = dummy2.predict(xtest)

8/8 [==============================] - 1s 21ms/step


In [53]:
prediction2 = np.argmax(prediction1,axis=1)
ac = accuracy_score(ytest,prediction2)
pres = precision_score(ytest,prediction2,average = 'macro')
f1 = f1_score(ytest,prediction2, average = 'macro')
rec = recall_score(ytest,prediction2, average = 'macro')

In [54]:
dataset2 = pd.DataFrame({
 'precision':[pres],
 'recall' :[rec],
 'accuracy':[ac],
 'F1_score':[f1]
})


In [55]:
dataset2

,precision,recall,accuracy,F1_score
0,0.244898,0.5,0.489796,0.328767


In [56]:
#GRU
dummy3 = Sequential()
em = Embedding(vocablen,100,input_length=max_len,trainable = True)
dummy3.add(em)
dummy3.add(GRU(100,return_sequences=True))
dummy3.add(Dense(2,activation='relu'))
dummy3.add(Flatten())
dummy3.add(Dense(1,activation='sigmoid'))
dummy3.compile(loss ="binary_crossentropy", optimizer = 'sgd', metrics=["accuracy"])

In [58]:
dummy3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 37, 100)           549800    
                                                                 
 gru (GRU)                   (None, 37, 100)           60600     
                                                                 
 dense_4 (Dense)             (None, 37, 2)             202       
                                                                 
 flatten_2 (Flatten)         (None, 74)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 75        
                                                                 
Total params: 610,677
Trainable params: 610,677
Non-trainable params: 0
_________________________________________________________________


In [59]:
dummy3.fit(xtrain,ytrain,batch_size=20,epochs = 10)

Epoch 1/10
37/37 [==============================] - 4s 38ms/step - loss: 0.6936 - accuracy: 0.4871
Epoch 2/10
37/37 [==============================] - 1s 38ms/step - loss: 0.6932 - accuracy: 0.5061
Epoch 3/10
37/37 [==============================] - 1s 38ms/step - loss: 0.6931 - accuracy: 0.5156
Epoch 4/10
37/37 [==============================] - 1s 37ms/step - loss: 0.6930 - accuracy: 0.5116
Epoch 5/10
37/37 [==============================] - 2s 43ms/step - loss: 0.6930 - accuracy: 0.5156
Epoch 6/10
37/37 [==============================] - 4s 98ms/step - loss: 0.6930 - accuracy: 0.5156
Epoch 7/10
37/37 [==============================] - 3s 71ms/step - loss: 0.6929 - accuracy: 0.5156
Epoch 8/10
37/37 [==============================] - 2s 50ms/step - loss: 0.6929 - accuracy: 0.5156
Epoch 9/10
37/37 [==============================] - 2s 66ms/step - loss: 0.6929 - accuracy: 0.5156
Epoch 10/10
37/37 [==============================] - 1s 40ms/step - loss: 0.6929 - accuracy: 0.5156


In [60]:
prediction1 = dummy3.predict(xtest)

8/8 [==============================] - 1s 10ms/step


In [61]:
prediction2 = np.argmax(prediction1,axis=1)
ac = accuracy_score(ytest,prediction2)
pres = precision_score(ytest,prediction2,average = 'macro')
f1 = f1_score(ytest,prediction2, average = 'macro')
rec = recall_score(ytest,prediction2, average = 'macro')

In [62]:
dummy3= pd.DataFrame({
 'precision':[pres],
 'recall' :[rec],
 'accuracy':[ac],
 'F1_score':[f1]
})



In [63]:
dummy3

,precision,recall,accuracy,F1_score
0,0.244898,0.5,0.489796,0.328767


In [64]:
#LSTM
dummy4 = Sequential()
em = Embedding(vocablen,100,input_length=max_len,trainable = True)
dummy4.add(em)
dummy4.add(LSTM(100,return_sequences=True))
dummy4.add(Dropout(0.7))
dummy4.add(Dense(2,activation='relu'))
dummy4.add(Flatten())
dummy4.add(Dense(1,activation='sigmoid'))
dummy4.compile(loss ="binary_crossentropy", optimizer = 'sgd', metrics=["accuracy"])

In [65]:
dummy4.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 37, 100)           549800    
                                                                 
 lstm_1 (LSTM)               (None, 37, 100)           80400     
                                                                 
 dropout_1 (Dropout)         (None, 37, 100)           0         
                                                                 
 dense_6 (Dense)             (None, 37, 2)             202       
                                                                 
 flatten_3 (Flatten)         (None, 74)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 75        
                                                                 
Total params: 630,477
Trainable params: 630,477
Non-tr

In [66]:
dummy4.fit(xtrain,ytrain,batch_size=20,epochs = 10)

Epoch 1/10
37/37 [==============================] - 4s 38ms/step - loss: 0.6937 - accuracy: 0.4884
Epoch 2/10
37/37 [==============================] - 2s 41ms/step - loss: 0.6930 - accuracy: 0.5075
Epoch 3/10
37/37 [==============================] - 2s 41ms/step - loss: 0.6931 - accuracy: 0.5048
Epoch 4/10
37/37 [==============================] - 2s 60ms/step - loss: 0.6925 - accuracy: 0.5252
Epoch 5/10
37/37 [==============================] - 4s 119ms/step - loss: 0.6923 - accuracy: 0.5374
Epoch 6/10
37/37 [==============================] - 4s 97ms/step - loss: 0.6929 - accuracy: 0.5075
Epoch 7/10
37/37 [==============================] - 3s 67ms/step - loss: 0.6929 - accuracy: 0.5238
Epoch 8/10
37/37 [==============================] - 2s 62ms/step - loss: 0.6932 - accuracy: 0.5129
Epoch 9/10
37/37 [==============================] - 3s 78ms/step - loss: 0.6927 - accuracy: 0.5211
Epoch 10/10
37/37 [==============================] - 2s 41ms/step - loss: 0.6929 - accuracy: 0.5170


In [ ]:
prediction1= dummy4.predict(xtest)

In [67]:
prediction2 = np.argmax(prediction1,axis=1)
ac = accuracy_score(ytest,prediction2)
pres = precision_score(ytest,prediction2,average = 'macro')
f1 = f1_score(ytest,prediction2, average = 'macro')
rec = recall_score(ytest,prediction2, average = 'macro')

In [68]:
dummy4 = pd.DataFrame({
 'precision':[pres],
 'recall' :[rec],
 'accuracy':[ac],
 'F1_score':[f1]
})



In [69]:
dummy4

,precision,recall,accuracy,F1_score
0,0.244898,0.5,0.489796,0.328767
